### Training

In [17]:
import os
import torch
import json
import glob
import time
import shutil
import random
from datasets import Dataset
from collections import defaultdict
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM, EarlyStoppingCallback, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

# 1. Config
LV_PATTERN = "../txt/latvian_sentences_*.txt"
GLOSS_PATTERN = "../txt/lsl_glosses_*.txt"
PRUNED_MODEL_PATH = "../models/mt5-pruned"
SAVE_DIR = "../models/mt5-lsl-model"

VAL_RATIO = 0.08
LOGGING_STEPS = 50
TRAIN_EPOCHS = 30
BATCH_SIZE = 8
LEARNING_RATE = 1e-3 # Try 3e-4 or 5e-4
EARLY_STOPPING_PATIENCE = 3
WEIGHT_DECAY = 0.01
MAX_NEW_TOKENS = 128
RARE_THRESHOLD = 2
OPTIMIZER = "adafactor"

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [18]:
# 2. Append all files to master list
lv_lines = []
gloss_lines = []
total_files = 0

for file_path in sorted(glob.glob(LV_PATTERN)):
    with open(file_path, "r", encoding="utf-8") as f:
        lv_lines.extend([line.strip() for line in f if line.strip()])
    total_files += 1

for file_path in sorted(glob.glob(GLOSS_PATTERN)):
    with open(file_path, "r", encoding="utf-8") as f:
        gloss_lines.extend([line.strip() for line in f if line.strip()])

assert len(lv_lines) == len(gloss_lines), f"❌ Mismatch! Sentence lines: {len(lv_lines)}, gloss lines: {len(gloss_lines)}"
print(f"✅ Loaded {total_files} batches with {len(lv_lines)} total pairs.")


# 3. Index
def get_next_run_dir(base_dir: str):
    os.makedirs(base_dir, exist_ok=True)

    existing = [
        int(d) for d in os.listdir(base_dir)
        if d.isdigit() and os.path.isdir(os.path.join(base_dir, d))
    ]

    next_id = max(existing, default=0) + 1
    run_dir = os.path.join(base_dir, f"{next_id}")
    os.makedirs(run_dir)

    return next_id, run_dir

RUN_ID, RUN_DIR = get_next_run_dir(SAVE_DIR)
print(f"📦 Training run #{RUN_ID} → {RUN_DIR}\n")


# 4. Create a dataset, Train (90%) and Test (10%) split
all_lv_text = " ".join(lv_lines)
all_gloss_text = " ".join(gloss_lines)
unique_lv_words = len(set(all_lv_text.split()))
unique_gloss_words = len(set(all_gloss_text.split()))

data = {"lv": lv_lines, "gloss": gloss_lines}

gloss_to_indices = defaultdict(list)

for idx, gloss_line in enumerate(gloss_lines):
    for gloss in gloss_line.split():
        gloss_to_indices[gloss].append(idx)

forced_train_indices = set()

for gloss, indices in gloss_to_indices.items():
    if len(indices) <= RARE_THRESHOLD:
        forced_train_indices.add(indices[0])

all_indices = set(range(len(lv_lines)))
remaining_indices = list(all_indices - forced_train_indices)

val_size = int(round(VAL_RATIO * len(remaining_indices)))

val_indices = set(random.sample(
    remaining_indices,
    min(val_size, len(remaining_indices))
))

train_indices = list(all_indices - val_indices)
val_indices = list(val_indices)

train_dataset = Dataset.from_dict({
    "lv": [lv_lines[i] for i in train_indices],
    "gloss": [gloss_lines[i] for i in train_indices],
})

val_dataset = Dataset.from_dict({
    "lv": [lv_lines[i] for i in val_indices],
    "gloss": [gloss_lines[i] for i in val_indices],
})

total = len(train_indices) + len(val_indices)

train_pct = 100 * len(train_indices) / total
val_pct = 100 * len(val_indices) / total

print(f"✅ Dataset split:")
print(f"  Train: {len(train_indices)} ({train_pct:.2f}%)")
print(f"  Validation: {len(val_indices)} ({val_pct:.2f}%)\n")


# 5. Check dataset validity
train_gloss_vocab = set()
for g in train_dataset["gloss"]:
    train_gloss_vocab.update(g.split())

missing = set(gloss_to_indices.keys()) - train_gloss_vocab
assert not missing, f"Missing glosses in training: {missing}"

single_occurrence_glosses = {
    gloss: indices
    for gloss, indices in gloss_to_indices.items()
    if len(indices) == 1
}

double_occurrence_glosses = {
    gloss: indices
    for gloss, indices in gloss_to_indices.items()
    if len(indices) == 2
}

print(f"Glosses appearing 1 time: {len(single_occurrence_glosses)}")
print(f"Glosses appearing 2 times: {len(double_occurrence_glosses)}")

if single_occurrence_glosses:
    print("Examples of 1 time glosses:")
    for g in list(single_occurrence_glosses.keys())[:5]:
        print("  ", g)

rare_val_examples = []
single_val_examples = []

for idx in val_indices:
    glosses = gloss_lines[idx].split()
    for g in glosses:
        freq = len(gloss_to_indices[g])

        if freq <= RARE_THRESHOLD:
            rare_val_examples.append((idx, g, freq))

        if freq == 1:
            single_val_examples.append((idx, g))

print(f"\nValidation samples containing rare glosses (≤{RARE_THRESHOLD}): {len(rare_val_examples)}")

if rare_val_examples:
    for idx, gloss, freq in rare_val_examples[:3]:
        print(f"  '{gloss}'")
        print(f"     LV:    {lv_lines[idx]}")
        print(f"     GLOSS: {gloss_lines[idx]}")

✅ Loaded 9 batches with 1914 total pairs.
📦 Training run #4 → ../models/mt5-lsl-model\4

✅ Dataset split:
  Train: 1782 (93.10%)
  Validation: 132 (6.90%)

Glosses appearing 1 time: 157
Glosses appearing 2 times: 179
Examples of 1 time glosses:
   atļauja
   steigties
   pieklājība
   miegs
   kleita

Validation samples containing rare glosses (≤2): 11
  'pārraut'
     LV:    Zāļu ražotāji informē Zāļu valsts aģentūru par zāļu piegāžu pārrāvumiem — kuras zāles nevarēs piegādāt un kad piegādi atjaunos.
     GLOSS: zāles ražot informācija zāles valsts iestāde par zāles piegāde pārraut kurš zāles nevarēt piegāde un kad piegāde atjaunot
  'skriet'
     LV:    Varbūt skriet, varbūt iet, bet varbūt braukt.
     GLOSS: varbūt skriet varbūt iet bet varbūt braukt
  'luksofors'
     LV:    Sarkanā, dzeltenā un zaļā luksofora krāsas.
     GLOSS: sarkans dzeltens zaļš luksofors krāsa


In [19]:
# 6. Load model
tokenizer = T5Tokenizer.from_pretrained(PRUNED_MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(PRUNED_MODEL_PATH)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Pruned Model loaded on: {device}")

# 7. Vocabulary remap
with open(os.path.join(PRUNED_MODEL_PATH, "vocab_map.json"), "r") as f:
    new2old_map = json.load(f)
    old2new_map = {v: int(k) for k, v in new2old_map.items()}

original_unk_id = tokenizer.unk_token_id
new_unk_id = old2new_map.get(original_unk_id, 0)

def remap_tokens(token_ids):
    return [old2new_map.get(tid, new_unk_id) for tid in token_ids]

def preprocess_function(examples):
    inputs = examples["lv"]
    model_inputs = tokenizer(inputs, max_length=MAX_NEW_TOKENS, truncation=True)

    labels = tokenizer(
        text_target=examples["gloss"],
        max_length=MAX_NEW_TOKENS,
        truncation=True
    )

    model_inputs["input_ids"] = [remap_tokens(ids) for ids in model_inputs["input_ids"]]
    model_inputs["labels"] = [remap_tokens(ids) for ids in labels["input_ids"]]
    
    return model_inputs

tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["lv", "gloss"]
)

tokenized_val = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["lv", "gloss"]
)

print(f"Remapped {len(old2new_map)} tokens.")

Pruned Model loaded on: cpu


Map:   0%|          | 0/1782 [00:00<?, ? examples/s]

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

Remapped 2974 tokens.


In [20]:
# 8. Training parameters
args = Seq2SeqTrainingArguments(
    output_dir=RUN_DIR,
    learning_rate=LEARNING_RATE,
    num_train_epochs=TRAIN_EPOCHS,

    eval_strategy="epoch",
    save_strategy="epoch",

    per_device_train_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    logging_steps=LOGGING_STEPS,
    save_total_limit=1,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    predict_with_generate=True,
    optim=OPTIMIZER,
    report_to="none",

    lr_scheduler_type="linear",
    warmup_ratio=0.1
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=EARLY_STOPPING_PATIENCE)]
)

C:\Users\liene\AppData\Local\Temp\ipykernel_11932\3517825860.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [21]:
# 9. Train (apprx. 28 mins)
start_time = time.time()
train_result = trainer.train()
end_time = time.time()

c:\Users\liene\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,7.241900,2.727356
2,2.160500,0.955298
3,1.264600,0.667086
4,0.903500,0.584804
5,0.639300,0.529693
6,0.503700,0.431287
7,0.412400,0.401445
8,0.297800,0.370321
9,0.277100,0.322822
10,0.209900,0.347867


c:\Users\liene\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\liene\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\liene\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\liene\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warn

In [22]:
# 10. Save model
model.save_pretrained(RUN_DIR)
tokenizer.save_pretrained(RUN_DIR)

print(f"Saved model to {RUN_DIR}")


# 11. Cleanup
print("Cleaning up temporary checkpoints...")
for item in os.listdir(RUN_DIR):
    item_path = os.path.join(RUN_DIR, item)
    if os.path.isdir(item_path) and item.startswith("checkpoint-"):
        try:
            shutil.rmtree(item_path)
            print(f"Deleted: {item}")
        except Exception as e:
            print(f"Could not delete {item}: {e}")

Saved model to ../models/mt5-lsl-model\4
Cleaning up temporary checkpoints...
Deleted: checkpoint-2676


In [23]:
# 12. Generate metadata

# 12a. Time
current_time = time.ctime()
training_duration = end_time - start_time

# 12b. Metrics of best saved model
best_epoch = None
best_train_loss = None
best_eval_loss = trainer.state.best_metric

best_checkpoint = trainer.state.best_model_checkpoint
best_step = int(best_checkpoint.split("-")[-1]) if best_checkpoint else None

for entry in trainer.state.log_history:
    if "eval_loss" in entry and abs(entry["eval_loss"] - best_eval_loss) < 1e-6:
        best_epoch = entry.get("epoch")

    if best_step is not None and "loss" in entry and entry["step"] <= best_step:
        best_train_loss = entry["loss"]

# 12c. How many epochs were trained
total_epoch = None
total_steps = train_result.global_step

for entry in trainer.state.log_history:
    if "epoch" in entry:
        total_epoch = int(entry["epoch"])

if total_epoch is None and trainer.state.log_history:
    last_log = trainer.state.log_history[-1]
    if 'epoch' in last_log:
        total_epoch = int(last_log['epoch'])

# 12d. Sentence examples
NUM_EXAMPLES = 6

train_example_indices = random.sample(train_indices, min(NUM_EXAMPLES, len(train_indices)))
val_example_indices = random.sample(val_indices, min(NUM_EXAMPLES, len(val_indices)))

def generate_examples(indices):
    results = []
    model.eval()

    for idx in indices:
        text = lv_lines[idx]

        inputs = tokenizer(
            text,
            return_tensors="pt",
            max_length=MAX_NEW_TOKENS,
            truncation=True
        )

        mapped_ids = [remap_tokens(ids) for ids in inputs.input_ids.tolist()]
        input_tensor = torch.tensor(mapped_ids).to(device)

        with torch.no_grad():
            outputs = model.generate(input_tensor, max_new_tokens=MAX_NEW_TOKENS)

        out_ids = outputs[0].tolist()
        orig_ids = [
            int(new2old_map.get(str(tid), tokenizer.unk_token_id))
            for tid in out_ids
        ]

        decoded = tokenizer.decode(orig_ids, skip_special_tokens=True)

        results.append({
            "input": text,
            "output": decoded,
            "target": gloss_lines[idx]
        })

    return results

train_sample_results = []
validation_sample_results = []
train_sample_results += generate_examples(train_example_indices)
validation_sample_results += generate_examples(val_example_indices)

# 12e. Metadata structure
metadata = {
    "general_info": {
        "id": RUN_ID,
        "seed": SEED,
        "base_model" : "google/mt5-small",
        "time_completed": current_time
    },
    "dataset_stats": {
        "total_pairs": len(lv_lines),
        "total_files": total_files,
        "unique_sentence_words": unique_lv_words,
        "unique_gloss_words": unique_gloss_words,
        "tokens_remapped": len(old2new_map),
    },
    "dataset_split": {
        "rare_gloss_threshold": RARE_THRESHOLD,
        "train_split": len(tokenized_train),
        "validation_split": len(tokenized_val),
        "train_split_%": f"{train_pct:.2f}%",
        "validation_split_%": f"{val_pct:.2f}%"
    },
    "generation": {
        "max_new_tokens": MAX_NEW_TOKENS,
        "decoder_start_token_id": model.config.decoder_start_token_id
    },
    "training_stats": {
        "duration_seconds": round(training_duration, 2),
        "epochs_completed": total_epoch,
        "batch_size": args.per_device_train_batch_size,
        "learning_rate": args.learning_rate,
        "weight_decay": args.weight_decay,
        "early_stopping_patience": EARLY_STOPPING_PATIENCE,
        "epochs_planned": args.num_train_epochs,
        "optimizer": OPTIMIZER,
        "mean_train_loss_over_all_steps": train_result.training_loss,
    },
    "final_model": {
        "epoch": best_epoch,
        "train_loss": best_train_loss,
        "eval_loss": best_eval_loss,
        "checkpoint_step": trainer.state.best_model_checkpoint.split("-")[-1] if trainer.state.best_model_checkpoint else total_steps
    },
    "test_samples": {
        "train": train_sample_results,
        "validation": validation_sample_results
    }
}

# 12f. Save to JSON
meta_path = os.path.join(RUN_DIR, "metadata.json")
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4, ensure_ascii=False)

print(f"✅ Metadata saved to: {meta_path}")


✅ Metadata saved to: ../models/mt5-lsl-model\4\metadata.json


## Testing

In [24]:
import torch
import json
import os

# --- 1. SETUP MAPS (Crucial!) ---
# We need to load the map to convert between "Big Tokenizer" and "Small Model"
vocab_map_path = os.path.join(PRUNED_MODEL_PATH, "vocab_map.json")

with open(vocab_map_path, "r") as f:
    new2old_map = json.load(f)
    # We need both directions!
    old2new_map = {int(v): int(k) for k, v in new2old_map.items()} # Big -> Small
    new2old_map = {int(k): int(v) for k, v in new2old_map.items()} # Small -> Big

original_unk_id = tokenizer.unk_token_id
pruned_unk_id = old2new_map.get(original_unk_id, 0) # Fallback to 0 if not found


# --- 2. CUSTOM TRANSLATION FUNCTION ---
def predict_gloss(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_NEW_TOKENS)
    input_ids = inputs.input_ids[0].tolist()
    
    # Remap to pruned IDs
    pruned_input_ids = [old2new_map.get(tid, pruned_unk_id) for tid in input_ids]
    input_tensor = torch.tensor([pruned_input_ids]).to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_tensor,
            max_length=32,
            num_beams=3,
            early_stopping=True,
            # no_repeat_ngram_size=2 # If we don't want many repeats!
        )
    
    # Remap output IDs back to full tokenizer
    output_ids = outputs[0].tolist()
    original_output_ids = [new2old_map.get(tid, tokenizer.unk_token_id) for tid in output_ids]

    # Decode
    return tokenizer.decode(original_output_ids, skip_special_tokens=True)


# --- 3. RUN TESTS ---
print("\n--- RESULTS ---")

test_sentences = [
    lv_lines[5],
    lv_lines[25],
    lv_lines[35],
    lv_lines[45],
    lv_lines[55]
]

for text in test_sentences:
    gloss = predict_gloss(text)
    print(f"\nInput:  {text}")
    print(f"Result: {gloss}")


--- RESULTS ---

Input:  Sveiks, [NAME].
Result: sveiks [NAME]

Input:  Kādā vārdā tevi saukt?
Result: kāds vārds tu saukt

Input:  [NAME] ir šīs personas vārds.
Result: [NAME] ir tas cilvēks vārds

Input:  Es negribu tevi redzēt...
Result: es negribēt tu redzēt

Input:  Novēlam Jums veiksmi!
Result: novēlēt tavs veiksme


In [25]:
import torch.nn.functional as F

model.eval()
sample_losses = []

for i in range(len(tokenized_val)):
    batch = tokenized_val[i]
    input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to(device)
    labels = torch.tensor(batch["labels"]).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=labels)

    loss = outputs.loss.item()
    sample_losses.append((i, loss))

hardest = sorted(sample_losses, key=lambda x: x[1], reverse=True)[:10]

for idx, loss in hardest:
    print(f"Loss: {loss:.3f}")
    print("LV:   ", val_dataset[idx]["lv"])
    print("GOLD: ", val_dataset[idx]["gloss"])
    print()

Loss: 2.164
LV:    Pulksteņa laiks ir divpadsmit.
GOLD:  pulkstenis laiks ir 12

Loss: 1.846
LV:    Mainīsim darbu.
GOLD:  mainīt darbs

Loss: 1.784
LV:    Dabas katastrofas arī citur Eiropas valstīs.
GOLD:  daba avārija arī cits eiropa valsts

Loss: 1.735
LV:    Nekas šausmīgs, paņemsim taksi.
GOLD:  nekas šausmas paņemt taksometrs

Loss: 1.657
LV:    Trīsdesmit pirmais augusts tiek uzskatīts par vasaras beigām.
GOLD:  30 pirmais augusts ir uzskatāms par vasara beigas

Loss: 1.582
LV:    Prieks tevi satikt.
GOLD:  prieks tu tikties

Loss: 1.138
LV:    Viņas nodarbojas ar zemkopību.
GOLD:  viņi nodarboties ar zeme kopt

Loss: 1.118
LV:    Sarkanā, dzeltenā un zaļā luksofora krāsas.
GOLD:  sarkans dzeltens zaļš luksofors krāsa

Loss: 0.986
LV:    Rīgas Dzemdību namā šajā nedēļā divas ģimenes sagaidījušas dvīnīšus.
GOLD:  rīga dzimt māja šodien nedēļa 2 ģimene gaidīt dvīnis

Loss: 0.968
LV:    Šorīt temperatūra man bija [NUM].
GOLD:  šodien rīts temperatūra_veselība bija [NUM]



In [26]:
from collections import Counter

missing_gloss = Counter()
extra_gloss = Counter()

for i in range(len(val_dataset)):
    src = val_dataset[i]["lv"]
    gold = val_dataset[i]["gloss"].split()

    inputs = tokenizer(src, return_tensors="pt", truncation=True).to(device)
    mapped_ids = [remap_tokens(ids) for ids in inputs.input_ids.tolist()]
    input_tensor = torch.tensor(mapped_ids).to(device)

    with torch.no_grad():
        out = model.generate(input_tensor, max_new_tokens=MAX_NEW_TOKENS)

    out_ids = out[0].tolist()
    orig_ids = [int(new2old_map.get(str(tid), tokenizer.unk_token_id)) for tid in out_ids]
    pred = tokenizer.decode(orig_ids, skip_special_tokens=True).split()

    gold_c = Counter(gold)
    pred_c = Counter(pred)

    # Missing tokens (under-produced)
    for g, cnt in (gold_c - pred_c).items():
        missing_gloss[g] += cnt

    # Extra tokens (over-produced / hallucinated)
    for g, cnt in (pred_c - gold_c).items():
        extra_gloss[g] += cnt

print("Most frequently missed glosses:")
print(missing_gloss.most_common(10))

print("\nMost frequently hallucinated glosses:")
print(extra_gloss.most_common(10))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Most frequently missed glosses:
[('ir', 26), ('es', 11), ('un', 10), ('ar', 9), ('tas', 9), ('tu', 8), ('labs', 8), ('mans', 8), ('vai', 8), ('[NUM]', 7)]

Most frequently hallucinated glosses:
[]
